In [1]:
import os

In [2]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\chicken-Disease-Classification\\research'

In [3]:
os.chdir("c:\\Users\\quamr\\OneDrive\\Desktop\\project\\chicken-Disease-Classification")

In [4]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import*
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        # create directory if it does not exist
        os.makedirs(
            os.path.dirname(self.config.local_data_file),
            exist_ok=True
        )

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! \n{headers}")
        else:
            logger.info("File already exists. Skipping download.")


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-10 20:39:57,050: INFO: common]: yaml file: config\config.yaml loaded successfully
[2026-01-10 20:39:57,054: INFO: common]: yaml file: params.yaml loaded successfully
[2026-01-10 20:39:57,055: INFO: common]: created directory at: artifacts
[2026-01-10 20:40:29,727: INFO: 2934248033]: artifacts\data_ingestion\data.zip downloaded! 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D413:1DB25A:2E419F:642C6E:69626BCC
Accept-Ranges: bytes
Date: Sat, 10 Jan 2026 15:10:06 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830060-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1768057805.988258,VS0,VE1025
Vary: Authorization,Accept-Encoding
Access-Co